# Многопоточность в Python

### Запуск потока с применением threading

In [1]:
import threading

class MyThread(threading.Thread):
    def __init__(self, x):
        threading.Thread.__init__(self)
        self.x = x

    def run(self):
        print('Starting processing %i...' % x)
        is_prime(self.x)

## Распределение задач между потоками при помощи очереди

Queue предоставляет нам механизм взаимодействия потоков между процессами FIFO (первым пришел — первым обслужен).
Ниже пример, где "воркеры" выполняют вычисление чисел Фибоначчи.
# Версия 1
Воркеры "добывают" себе задание из очереди и печатают результат на экран.

In [2]:
from multiprocessing import Process, Queue
import os
import time

workers_number = 4
final_fibonacci_number = 40


def worker(tasks: Queue, process_index: int):
    def fib(n: int) -> int:
        return fib(n-1) + fib(n-2) if n > 2 else 1
    
    while not tasks.empty():  # пока очередь не пуста выполняем одну очередную задачу
        number = tasks.get()
        answer = fib(number)
        print(f"worker {process_index}, PID={os.getpid()}: fib({number}) = {answer}")
    

def main():
    tasks = Queue()
    for n in range(0, final_fibonacci_number + 1):
        tasks.put(n)

    workers = []
    for process_index in range(workers_number):
        worker_process = Process(target = worker, args = (tasks, process_index,))
        workers.append(worker_process)
    print("Parent process queued all the tasks.")
    start_time = time.perf_counter()
    for worker_process in workers:
        worker_process.start()
    print("Parent process started workers processes.")
    for worker_process in workers:
        worker_process.join()
    finish_time = time.perf_counter()
    print("Parent process joined all the workers processes. Duration:",
          finish_time - start_time)
    
    
if __name__ == '__main__':
    main()

Parent process queued all the tasks.
worker 2, PID=27966: fib(1) = 1
worker 3, PID=27967: fib(3) = 2
worker 1, PID=27963: fib(2) = 1
worker 1, PID=27963: fib(5) = 5
worker 1, PID=27963: fib(6) = 8
worker 0, PID=27962: fib(0) = 1
worker 2, PID=27966: fib(4) = 3
worker 0, PID=27962: fib(9) = 34
worker 1, PID=27963: fib(8) = 21
worker 3, PID=27967: fib(7) = 13
worker 2, PID=27966: fib(10) = 55
worker 1, PID=27963: fib(11) = 89
worker 3, PID=27967: fib(12) = 144
worker 2, PID=27966: fib(13) = 233
worker 2, PID=27966: fib(14) = 377
worker 3, PID=27967: fib(15) = 610
worker 2, PID=27966: fib(16) = 987
worker 3, PID=27967: fib(17) = 1597
worker 0, PID=27962: fib(19) = 4181
worker 1, PID=27963: fib(20) = 6765
worker 0, PID=27962: fib(21) = 10946
worker 2, PID=27966: fib(18) = 2584
worker 1, PID=27963: fib(22) = 17711
worker 3, PID=27967: fib(23) = 28657
worker 0, PID=27962: fib(24) = 46368
worker 2, PID=27966: fib(25) = 75025
worker 3, PID=27967: fib(27) = 196418
worker 1, PID=27963: fib(26) =

# Версия 2
Теперь не будем печатать результаты на экран, но сделаем очередь результатов:

In [3]:
from multiprocessing import Process, Queue
import os
import time

workers_number = 4
final_fibonacci_number = 40


def worker(tasks: Queue, answers: Queue, process_index: int):
    def fib(n: int) -> int:
        return fib(n-1) + fib(n-2) if n > 2 else 1
    
    while not tasks.empty():  # пока очередь не пуста выполняем одну очередную задачу
        number = tasks.get()
        answer = fib(number)
        answers.put((process_index, os.getpid(), number, answer,))
        #print(f"worker {process_index}, PID={os.getpid()}: fib({number}) = {answer}")
    

def main():
    tasks = Queue()
    answers = Queue()
    for n in range(0, final_fibonacci_number + 1):
        tasks.put(n)

    workers = []
    for process_index in range(workers_number):
        worker_process = Process(target = worker,
                                 args = (tasks, answers, process_index,))
        workers.append(worker_process)
    print("Parent process queued all the tasks.")
    start_time = time.perf_counter()
    for worker_process in workers:
        worker_process.start()
    print("Parent process started workers processes.")
    for worker_process in workers:
        worker_process.join()
    # Всё, тут мы вышли из режима многозадачности. Работает один родительский процесс.
    finish_time = time.perf_counter()
    print("Parent process joined all the workers processes. Duration:",
          finish_time - start_time)
    # Отладочная распечатка результатов
    ordered_answers = []
    while not answers.empty():
        process_index, PID, number, answer = answers.get()
        ordered_answers.append((number, answer,))
        print(f"worker {process_index}, PID={PID}: fib({number}) = {answer}")
    # Красивая распечатка полученных результатов:
    ordered_answers.sort()
    print(*(answer for number, answer in ordered_answers))
    
if __name__ == '__main__':
    main()

Parent process queued all the tasks.
Parent process started workers processes.
Parent process joined all the workers processes. Duration: 39.01372305800032
worker 0, PID=28102: fib(0) = 1
worker 0, PID=28102: fib(1) = 1
worker 0, PID=28102: fib(2) = 1
worker 0, PID=28102: fib(3) = 2
worker 0, PID=28102: fib(4) = 3
worker 0, PID=28102: fib(5) = 5
worker 1, PID=28103: fib(17) = 1597
worker 0, PID=28102: fib(6) = 8
worker 0, PID=28102: fib(7) = 13
worker 0, PID=28102: fib(8) = 21
worker 0, PID=28102: fib(9) = 34
worker 0, PID=28102: fib(10) = 55
worker 0, PID=28102: fib(11) = 89
worker 0, PID=28102: fib(12) = 144
worker 0, PID=28102: fib(13) = 233
worker 0, PID=28102: fib(14) = 377
worker 0, PID=28102: fib(15) = 610
worker 0, PID=28102: fib(16) = 987
worker 0, PID=28102: fib(18) = 2584
worker 0, PID=28102: fib(19) = 4181
worker 0, PID=28102: fib(21) = 10946
worker 2, PID=28105: fib(20) = 6765
worker 2, PID=28105: fib(22) = 17711
worker 0, PID=28102: fib(24) = 46368
worker 3, PID=28107: fi

# Версия 3
Откажемся от явного использования Process, да и от Queue тоже, в пользу Pool.

In [4]:
from multiprocessing import Pool
import os
import time

workers_number = 4
final_fibonacci_number = 40


def fib(n: int) -> int:
    return fib(n-1) + fib(n-2) if n > 2 else 1
    

def main():
    tasks = list(range(0, final_fibonacci_number))
    start_time = time.perf_counter()
    # Уход в паралельные вычисления:
    with Pool(workers_number) as pool_of_processes:
        answers = list(pool_of_processes.map(fib, tasks))
    # Всё, тут мы вышли из режима многозадачности. Работает один родительский процесс.
    finish_time = time.perf_counter()
    print("Duration:", finish_time - start_time)
    
    print(*answers)
    
if __name__ == '__main__':
    main()

Duration: 20.047907690999637
1 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986


In [5]:
import time

final_fibonacci_number = 40


def fib(n: int) -> int:
    return fib(n-1) + fib(n-2) if n > 2 else 1
    

def main():
    tasks = list(range(0, final_fibonacci_number + 1))
    start_time = time.perf_counter()
    # Никаких параллельных вычислений! Работаем сами:
    answers = []
    for number in tasks:
        answers.append(fib(number))
    # Работает один родительский процесс по-прежнему.
    finish_time = time.perf_counter()
    print("Duration:", finish_time - start_time)
    
    print(*answers)
    
if __name__ == '__main__':
    main()

Duration: 59.498384865000844
1 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946 17711 28657 46368 75025 121393 196418 317811 514229 832040 1346269 2178309 3524578 5702887 9227465 14930352 24157817 39088169 63245986 102334155
